# Data Quality Visualizations

In [ ]:
import time
import boto3
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import contextily as cx

# Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client("s3")  # for lower-level processes

# Set relative paths to other folders and objects in repository.
BUCKET_NAME = "wecc-historical-wx"
QAQC_DIR = "3_qaqc_wx"
MERGE_DIR = "4_merge_wx"
stations_csv_path = f"s3://{BUCKET_NAME}/{QAQC_DIR}/all_network_stationlist_qaqc.csv"
shapepath = "s3://wecc-historical-wx/0_maps/tl_2021_us_state"

### Merge with station list

In [ ]:
station_list = pd.read_csv(stations_csv_path)
sub_station_list = station_list[station_list['network']==network]

In [ ]:
sub_station_list

In [ ]:
merged_list = sub_station_list.merge(flag_rate_df, on="era-id")

In [ ]:
merged_list

next steps: extract station name

In [ ]:
network_flag_path_ = f"s3://{BUCKET_NAME}/{MERGE_DIR}/per_network_flag_counts_native_timestep/VCAPCD_flag_counts_native_timestep.csv"
flag_df = pd.read_csv(network_flag_path)

## Final functions

In [ ]:
def _pairwise_rate(flag_df: pd.DataFrame, running_rate_df, station_name: str) -> pd.DataFrame:
    """
    Generates flag rates dataframe for input flag counts datafeame and then adds it to the running flag rate dataframe.
    Helper function for network_rate_tables() and station_rate_table().

    Parameters
    ----------
    flag_df: pd.DataFrame
        flag rates dataframe for next station
    running_rate_df: pd.DataFrame
        dataframe of previously added station flag rates

    Returns
    -------
    rates_df_merged: pd.DataFrame

    """
    # Make the eraqc_flag_values column the index
    flag_df = flag_df.set_index("eraqc_flag_values")

    # Count up the flagged observations - so counts in all but the "no_flag" and "total_obs_count" rows
    subset = flag_df[~flag_df.index.isin(["no_flag", "total_obs_count"])]
    totals = subset.sum(numeric_only=True)
    flag_df.loc["total_flag"] = pd.Series(totals)

    # And then use those total to calculate the per-variable flag rates
    frac = flag_df.loc["total_flag"] / flag_df.loc["total_obs_count"]
    flag_df.loc["flag_rate"] = pd.Series(frac)

    # Keep only the rate
    rates_df = flag_df.loc[["flag_rate"]]
    rates_df = rates_df.rename(index={"flag_rate": station_name})

    rates_df = rates_df.reset_index()

    # Finally, append column of total observation count
    flag_df = flag_df.reset_index()
    total_obs = flag_df[flag_df['eraqc_flag_values']=='total_obs_count'].iloc[0,1]
    rates_df['total_obs_count'] = total_obs

    if len(running_rate_df) == 0:
        return rates_df

    else:
        rates_df_merged = pd.merge(rates_df, running_rate_df, how="outer")
        return rates_df_merged

In [35]:
def network_rate_tables() -> None:
    """
    Generates flag rates tables at either the station or network level.

    Parameters
    ----------
    flag_df_1: pd.DataFrame
        dataframe of previously summed station flag counts
    flag_df_2: pd.DataFrame
        flag counts dataframes for next station

    Returns
    -------
    summed_df: pd.DataFrame
    """

    # the function iteratively adds in flag counts to this dataframe
    flag_rate_df = []

    # point to folder containing station flag count CSVs
    flags_prefix = f"{MERGE_DIR}/per_network_flag_counts_native_timestep"

    ## Merge flag counts

    # loop through all CSVs are the given level
    for item in s3.Bucket(BUCKET_NAME).objects.filter(Prefix=flags_prefix):
        obj = s3_cl.get_object(Bucket=BUCKET_NAME, Key=item.key)
        flags = pd.read_csv(obj["Body"])
        station_name = item.key.split(flags_prefix + "/")[1].split("_flag")[0]
        # the CSV is empty
        if flags.empty:
            continue
        # the CSV is not empty
        else:
            # remove "QAQC_function" and "Flag_meaning" columns
            flags = flags.drop(["QAQC_function", "Flag_meaning"], axis=1)

            # send current dataframe and dataframe of previously summed counts to helper function
            flag_rate_df = _pairwise_rate(flags, flag_rate_df, station_name)
    
    # change "eraqc_flag_values" to "stations"
    flag_rate_df = flag_rate_df.rename(columns={"eraqc_flag_values": "networks"})

    ## Send final flag rates file to AWS as CSV
    csv_s3_filepath = f"s3://wecc-historical-wx/4_merge_wx/network_flag_rates.csv"
    # flag_rate_df.to_csv(csv_s3_filepath, index=False)
    print(f"Sending station flag rates CSV to: {csv_s3_filepath}")

    return flag_rate_df

In [33]:
test = network_rate_tables()

Sending station flag rates CSV to: s3://wecc-historical-wx/4_merge_wx/network_flag_rates.csv


In [34]:
test

,networks,elevation,hurs,pr_1h,pr,ps,rsds,sfcWind_dir,sfcWind,tas,...,ps_altimeter,ps_derived,accum_pr_24h,accum_pr_localmid,pr_24h,pr_5min,pr_localmid,accum_pr_1h,accum_pr_5min,pvp_derived
0,VCAPCD,0.000000,1.586439e-03,0.000082,0.000000,5.987326e-04,2.429234e-03,0.003154,0.001392,0.002116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SNOTEL,0.000000,1.435126e-04,NaN,0.321078,NaN,3.075973e-02,NaN,NaN,0.002551,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SHASAVAL,0.000000,2.139301e-04,0.000131,NaN,NaN,NaN,0.011551,0.007433,0.000178,...,0.002454,0.002454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SGXWFO,0.000000,2.127545e-08,0.001365,0.000005,9.070196e-05,1.120008e-02,0.040221,0.093519,0.001410,...,0.353959,0.336662,1.0,1.0,0.060711,0.000000,0.001603,1.0,NaN,NaN
4,SCAN,0.000000,3.392482e-05,NaN,0.006791,NaN,NaN,0.003057,NaN,0.000539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,RAWS,0.000000,4.626154e-06,0.000034,0.017633,4.855570e-02,1.626169e-02,0.095958,0.000583,0.002110,...,0.066155,0.061133,1.0,NaN,0.000089,0.000000,NaN,1.0,NaN,NaN
6,OtherISD,0.019282,NaN,NaN,0.001198,1.851700e-02,0.000000e+00,0.000392,0.000506,0.000576,...,0.059712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NOS-PORTS,0.000000,NaN,NaN,0.000000,NaN,1.047805e-07,0.003711,0.155035,0.000288,...,0.084542,0.084627,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
8,NOS-NWLON,0.000000,5.414964e-07,0.349142,0.000008,9.077145e-07,4.746670e-08,0.004627,0.073874,0.000129,...,0.067260,0.067319,NaN,NaN,0.000128,0.290638,NaN,NaN,NaN,NaN
9,NDBC,0.070599,NaN,NaN,NaN,1.042413e-02,NaN,0.006146,0.001205,0.032881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
test2 = test.rename(columns={"eraqc_flag_values": "networks"})

In [27]:
test2

,networks,elevation,hurs,pr_1h,pr,ps,rsds,sfcWind_dir,sfcWind,tas,...,ps_altimeter,ps_derived,accum_pr_24h,accum_pr_localmid,pr_24h,pr_5min,pr_localmid,accum_pr_1h,accum_pr_5min,pvp_derived
0,VCAPCD,0.000000,1.586439e-03,0.000082,0.000000,5.987326e-04,2.429234e-03,0.003154,0.001392,0.002116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SNOTEL,0.000000,1.435126e-04,NaN,0.321078,NaN,3.075973e-02,NaN,NaN,0.002551,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SHASAVAL,0.000000,2.139301e-04,0.000131,NaN,NaN,NaN,0.011551,0.007433,0.000178,...,0.002454,0.002454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SGXWFO,0.000000,2.127545e-08,0.001365,0.000005,9.070196e-05,1.120008e-02,0.040221,0.093519,0.001410,...,0.353959,0.336662,1.0,1.0,0.060711,0.000000,0.001603,1.0,NaN,NaN
4,SCAN,0.000000,3.392482e-05,NaN,0.006791,NaN,NaN,0.003057,NaN,0.000539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,RAWS,0.000000,4.626154e-06,0.000034,0.017633,4.855570e-02,1.626169e-02,0.095958,0.000583,0.002110,...,0.066155,0.061133,1.0,NaN,0.000089,0.000000,NaN,1.0,NaN,NaN
6,OtherISD,0.019282,NaN,NaN,0.001198,1.851700e-02,0.000000e+00,0.000392,0.000506,0.000576,...,0.059712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NOS-PORTS,0.000000,NaN,NaN,0.000000,NaN,1.047805e-07,0.003711,0.155035,0.000288,...,0.084542,0.084627,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
8,NOS-NWLON,0.000000,5.414964e-07,0.349142,0.000008,9.077145e-07,4.746670e-08,0.004627,0.073874,0.000129,...,0.067260,0.067319,NaN,NaN,0.000128,0.290638,NaN,NaN,NaN,NaN
9,NDBC,0.070599,NaN,NaN,NaN,1.042413e-02,NaN,0.006146,0.001205,0.032881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
network_flag_path = f"s3://{BUCKET_NAME}/{MERGE_DIR}/per_network_flag_counts_native_timestep/VCAPCD_flag_counts_native_timestep.csv"
flag_df_1 = pd.read_csv(network_flag_path)

In [9]:
flag_df_1.drop(['QAQC_function','Flag_meaning'],axis=1)

,eraqc_flag_values,elevation,hurs,pr_1h,pr,ps,rsds,sfcWind_dir,sfcWind,tas,tdps_derived
0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0
5,6,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,0,0,0,0,0,0,0
7,8,0,0,0,0,0,0,0,0,0,0
8,9,0,0,0,0,0,0,0,0,0,0
9,10,0,0,0,0,0,0,0,0,0,0


In [10]:
flag_df_1

,eraqc_flag_values,elevation,hurs,pr_1h,pr,ps,rsds,sfcWind_dir,sfcWind,tas,tdps_derived,QAQC_function,Flag_meaning
0,1,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Suspect observation (i.e. buoy reports wind du...
1,2,0,0,0,0,0,0,0,0,0,0,spurious_buoy_check,Out of range for station official data tempora...
2,3,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from DEM (USGS 3DEP)
3,4,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation infilled from station
4,5,0,0,0,0,0,0,0,0,0,0,qaqc_elev_infill,Elevation manually infilled to be 0.0 m; occur...
5,6,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height missing
6,7,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_t,INACTIVE FLAG: Thermometer height not 2 meters
7,8,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height missing
8,9,0,0,0,0,0,0,0,0,0,0,qaqc_sensor_height_w,INACTIVE FLAG: Anemometer height not 10 meters
9,10,0,0,0,0,0,0,0,0,0,0,qaqc_precip_logic_nonegvals,Precipitation value reported below 0 (negative...


In [ ]:
def station_rate_tables() -> None:
    """
    Generates flag rates tables for each station across all networks.

    Parameters
    ----------
    flag_df_1: pd.DataFrame
        dataframe of previously summed station flag counts
    flag_df_2: pd.DataFrame
        flag counts dataframes for next station

    Returns
    -------
    summed_df: pd.DataFrame
    """
    network_list = ['VCAPCD','CDEC']

    # the function iteratively adds in flag counts to this dataframe
    flag_rate_df = []

    for network in network_list:
        # point to folder containing station flag count CSVs
        flags_prefix = f"{MERGE_DIR}/{network}/eraqc_counts_native_timestep"

        ## Merge flag counts

        # loop through all CSVs are the given level
        for item in s3.Bucket(BUCKET_NAME).objects.filter(Prefix=flags_prefix):
            obj = s3_cl.get_object(Bucket=BUCKET_NAME, Key=item.key)
            flags = pd.read_csv(obj["Body"])
            station_name = item.key.split(flags_prefix + "/")[1].split("_flag")[0]
            # the CSV is empty
            if flags.empty:
                continue
            # the CSV is not empty
            else:
                # send current dataframe and dataframe of previously summed counts to helper function
                flag_rate_df = _pairwise_rate(flags, flag_rate_df, station_name)

                flag_df['erac_']

        ## Send final flag rates file to AWS as CSV
        csv_s3_filepath = f"s3://wecc-historical-wx/4_merge_wx/station_flag_rates.csv"
        # flag_rate_df.to_csv(csv_s3_filepath, index=False)
        print(f"Sending station flag rates CSV to: {csv_s3_filepath}")

    return flag_rate_df

In [ ]:
station_rate_tables()

## Map

In [ ]:
map_list = merged_list

# Format dates in datetime format (this gets lost in import).
map_list["start-date"] = pd.to_datetime(map_list["start-date"], utc=True)
map_list["end-date"] = pd.to_datetime(map_list["end-date"], utc=True)

# Make a geodataframe.
gdf = gpd.GeoDataFrame(
    map_list,
    geometry=gpd.points_from_xy(map_list.longitude, map_list.latitude),
)
gdf.set_crs(epsg=4326, inplace=True)  # Set CRS

# Project data to match base tiles.
gdf_wm = gdf.to_crs(epsg=3857)  # Web mercator

# Read in geometry of continental US.
us = gpd.read_file(shapepath)

# Remove territories, AK, HI
rem_list = ["HI", "AK", "MP", "GU", "AS", "PR", "VI"]
us = us.loc[us.STUSPS.isin(rem_list) == False]

# Use to clip stations
us = us.to_crs(epsg=3857)
gdf_us = gdf_wm.clip(us)



In [ ]:
# Plot
ax = gdf_us.plot(
    "tas",
    figsize=(15, 15),
    alpha=1,
    markersize=3,
    legend=True,
    cmap="nipy_spectral",
)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
ax.set_axis_off()